In [1]:
# Imports previos

import matplotlib.pyplot as plt
import nltk
import networkx as nx

from Bio import Medline
from Bio import Entrez

import urllib.request

In [2]:
# Funciones necesarias:

def extract_key_from_value(dict, query):
    for key, values in dict.items():
        for synonym in values:
            if query == synonym:
                return key


# Database para enfermedades

**PLAN RAW PARA ENFERMEDADES**: en Disease Ontology, hay una lista<sup>1</sup> que contiene un ID de la enfermedad, con el id, nombre de la enfermedad y sinónimos. La idea es generar un diccionario  {key: ID y values: nombres de la enfermedad} y si se encuentra en un abstract la enfermedad (es decir, el value), cambiarlo por el ID (la key)
 
 <sup>1</sup>https://github.com/DiseaseOntology/HumanDiseaseOntology/blob/main/src/ontology/HumanDO.obo
 
*Nota*: https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo es el raw

In [21]:
# Importación de lista de enfermedades 

# Se inicializa el diccionario y las variables (inicialmente vacias y false)
disease_dict={}
disease_key=False
disease_name=[]

# Se abre la página del texto raw, se pone todo en minúsculas y se aplica el decode para que no haya problemas
obo=urllib.request.urlopen('https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/main/src/ontology/HumanDO.obo')
obo=[line.lower().decode("utf-8") for line in obo]

# Para cada linea del texto:

# Si empieza con [term] y NO hay definidos una key y value para el diccionario, se inicializan las variables.
# Si empieza con [term] y SÍ hay definidos una key y value para el diccionario, se añaden al diccionario y se resetean

# Si empieza con id, lo convierte en key (disease_key), quitando el "id:", el salto de línea y los espacios en blanco
# Si empieza con name o synonym, lo añade a la lista de values (disease_name) tras eliminar los saltos de línea, el 
#     synonym, el exact [] en el caso de los synonyms, y las comillas.

# Si empieza con [typedef], significa que ya no hay más terms, y que no interesa seguir parseando, con lo que se añade
# el último key-value y se termina

for line in obo:
            
    if line.startswith("[term]"):
        if disease_key and disease_name:
            disease_dict[disease_key]=disease_name
        disease_key=str()
        disease_name=list()
    
    if line.startswith("id"):
        disease_key=line.strip("id:").strip("\n").strip()

    elif line.startswith("name"):
        disease_name.append(line.strip("name:").strip("\n").strip())
    
    elif line.startswith("synonym"):
        disease_name.append(line.strip("\n").strip("synonym: ").strip( "exact []").replace('"',""))

    elif line.startswith("[typedef]"):
        disease_dict[disease_key] = disease_name
        break


In [24]:
print(f'El diccionario de enfermedades contiene {len(disease_dict)} entradas.')
# El resultado es 13046 para la versión del 22 de diciembre de 2020

El diccionario de enfermedades contiene 13046 entradas.


In [23]:
for key,values in disease_dict.items():
    print(f'{key} = {values}')

doid:0001816 = ['angiosarcoma', 'hemangiosarcoma']
doid:0002116 = ['pterygium', "surfer's eye"]
doid:0014667 = ['disease of metabolism', 'metabolic disease']
doid:0040001 = ['shrimp allergy']
doid:0040002 = ['aspirin allergy', 'acetylsalicylic acid allergy', 'asa allergy']
doid:0040003 = ['benzylpenicillin allergy', 'benzyl benicillin allergy', 'penicillin g allergy']
doid:0040004 = ['amoxicillin allergy']
doid:0040005 = ['ceftriaxone allergy', 'rocephin allergy']
doid:0040006 = ['carbamazepine allergy', 'carbamazepen allergy', 'tegretol allergy']
doid:0040007 = ['abacavir allergy', 'abc allergy']
doid:0040008 = ['isoniazide allergy', 'inh allergy', 'isonicotinylhydrazide allergy']
doid:0040009 = ['lidocaine allergy', 'lidoderm allergy', 'lignocaine allergy', 'xylocaine allergy']
doid:0040010 = ['mepivacaine allergy', 'carbocaine allergy', 'polocaine allergy']
doid:0040011 = ['phenobarbital allergy', 'luminal allergy', 'phenobarbitol allergy', 'phenobarbitone allergy']
doid:0040012 = [

doid:0060792 = ['hypomyelinating leukodystrophy 11', 'hld11']
doid:0060793 = ['hypomyelinating leukodystrophy 5', 'hld5', 'hypomyelination-congenital cataract syndrome']
doid:0060794 = ['hypomyelinating leukodystrophy 7 with or without oligodontia and-or hypogonadotropic hypogonadism', 'ataxia-delayed dentition-hypomyelination syndrome; odontoleukodystrophy', 'dentoleukoencephalopathy', 'hld7', 'hypomyelination-cerebellar atrophy-hypoplasia of the corpus callosum syndrome', 'leukodystrophy with oligodontia', 'leukoencephalopathy-ataxia-hypodontia-hypomyelination syndrome', 'tach syndrome', 'tremor-ataxia-central hypomyelination syndrome']
doid:0060795 = ['hypomyelinating leukodystrophy 13', 'hld13']
doid:0060796 = ['hypomyelinating leukodystrophy 12', 'hld12']
doid:0060797 = ['hypomyelinating leukodystrophy 8 with or without oligodontia and-or hypogonadotropic hypogonadism', 'hld8']
doid:0060798 = ['hypomyelinating leukodystrophy 6', 'h-abc', 'habc', 'hld6', 'hypomyelinating leukodystr

doid:0110556 = ['autosomal dominant nonsyndromic deafness 27', 'autosomal dominant deafness 27', 'dfna27']
doid:0110557 = ['autosomal dominant nonsyndromic deafness 28', 'autosomal dominant deafness 28', 'dfna28']
doid:0110558 = ['autosomal dominant nonsyndromic deafness 2a', 'autosomal dominant deafness 2a', 'dfna2a']
doid:0110559 = ['autosomal dominant nonsyndromic deafness 2b', 'autosomal dominant deafness 2b', 'dfna2b']
doid:0110560 = ['autosomal dominant nonsyndromic deafness 30', 'autosomal dominant deafness 30', 'dfna30']
doid:0110561 = ['autosomal dominant nonsyndromic deafness 31', 'autosomal dominant deafness 31', 'dfna31']
doid:0110562 = ['autosomal dominant nonsyndromic deafness 33', 'autosomal dominant deafness 33', 'dfna33']
doid:0110563 = ['autosomal dominant nonsyndromic deafness 36', 'autosomal dominant deafness 36', 'dfna36']
doid:0110564 = ['autosomal dominant nonsyndromic deafness 3a', 'autosomal dominant deafness 3a', 'dfna3a']
doid:0110565 = ['autosomal dominant n

doid:10561 = ['obsolete yaws hyperkeratosis', 'hyperkeratosis due to yaws', 'hyperkeratosis of yaws (disorder)']
doid:10564 = ['obsolete gangosa of yaws', 'gangosa', 'gangosa of yaws (disorder)', 'rhinopharyngitis mutilans']
doid:10567 = ['late yaws', 'gummata and ulcers due to yaws', 'gummata of yaws', 'gummatous frambeside', 'multiple papillomata due to yaws and wet crab yaws', 'nodular late yaws', 'ulcers of yaws', 'yaws gummata and ulcers']
doid:10568 = ['early yaws', 'bone and joint lesions due to yaws', 'bone and joint yaws lesion', 'chancre of yaws', 'frambesia, initial or primary', 'initial frambesial ulcer', 'initial lesions of yaws', 'primary frambesia']
doid:10569 = ['obsolete myopathy of critical illness', 'critical illness myopathy']
doid:1057 = ['obsolete inborn amino acid transport disorder']
doid:10573 = ['osteomalacia']
doid:10574 = ['obsolete vitamin d deficiency', 'avitaminosis d', 'avitaminosis d nos (disorder)', 'deficiency of vitamin d (disorder)', 'unspecified vi

doid:178 = ['vascular disease', 'vascular tissue disease']
doid:1781 = ['thyroid gland cancer', 'malignant neoplasm of thyroid gland', 'malignant tumour of thyroid gland', 'neoplasm of thyroid gland', 'thyroid gland neoplasm', 'thyroid neoplasm']
doid:1785 = ['pituitary cancer', 'malignant pituitary neoplasm', 'malignant tumor of pituitary gland', 'neoplasm of pituitary gland', 'pituitary gland cancer', 'pituitary gland neoplasm', 'pituitary neoplasm']
doid:1786 = ['adrenal rest tumor', 'adrenal rest neoplasm']
doid:1787 = ['pericarditis']
doid:1788 = ['peritoneal mesothelioma', 'advanced malignant mesothelioma of peritoneum', 'advanced malignant peritoneal mesothelioma', 'malignant mesothelioma of peritoneum']
doid:1789 = ['benign peritoneal mesothelioma', 'mesothelioma of peritoneum']
doid:1790 = ['malignant mesothelioma', 'advanced malignant mesothelioma', 'asbestos-related malignant mesothelioma', 'diffuse malignant mesothelioma', 'malignant tumor of mesothelium']
doid:1791 = ['per

doid:4786 = ['obsolete primitive neuroectodermal tumor with leptomeningeal spread']
doid:4787 = ['cerebellopontine angle primitive neuroectodermal tumor', 'cerebellopontine angle pnet']
doid:4788 = ['intracranial primitive neuroectodermal tumor', 'intracranial pnet']
doid:479 = ['angiokeratoma', 'angiokeratoma', 'angiokeratoma of skin', 'cutaneous angiokeratoma', 'skin angiokeratoma']
doid:4790 = ['medulloepithelioma', 'diktyoma, malignant', 'medulloepithelioma', 'medulloepithelioma, central nervous system']
doid:4791 = ['supratentorial primitive neuroectodermal tumor', 'supratentorial pnet']
doid:4794 = ['ependymoblastoma']
doid:4795 = ['gm2 gangliosidosis, ab variant', 'tay-sachs disease ab variant', 'tay-sachs disease, variant ab']
doid:4796 = ['space motion sickness']
doid:4797 = ['sm-ahnmd', 'systemic mastocytosis with associated clonal haematological non-mast cell lineage disease', 'systemic mastocytosis with associated clonal, hematologic non-mast-cell lineage disease']
doid:479

doid:8146 = ['obsolete anaplastic secondary supratentorial astrocytic neoplasm', 'undifferentiated secondary supratentorial astrocytoma']
doid:8148 = ['obsolete recurrent childhood malignant germ cell tumor', 'recurrent childhood malignant germ cell neoplasm']
doid:8149 = ['malignant childhood germ cell neoplasm', 'malignant pediatric germ cell tumor']
doid:8150 = ['pancreatic invasive intraductal papillary-mucinous carcinoma']
doid:8151 = ['appendiceal l-cell glucagon-like peptide producing tumor', 'appendiceal l-cell glucagon-like peptide producing tumour', 'malignant appendiceal glucagonoma', 'malignant appendiceal l-cell glucagon-like peptide producing tumor', 'malignant appendiceal l-cell glucagon-like peptide producing tumour']
doid:8152 = ['obsolete distal 1/3 of common bile duct adenocarcinoma', 'adenocarcinoma of the distal 1/3 of the common bile duct']
doid:8153 = ['fibroosseous pseudotumor of digits', 'fibroosseous digital pseudotumor']
doid:8158 = ['complement component 5 d

# Database para bacterias

**PLAN RAW PARA BACTERIAS**:  Se obtendrá una lista con el nombre científico de las bacterias, tanto con nombre genérico como epíteto específico (ojo con pasarla a  minúsculas) en un solo string. Una opción es generar un diccionario {key: Nombre científico completo pero todo junto y values: nombre científico separado y nombre abreviado}, y repetir el proceso anterior. Así, todos los nombres científicos (staphylococcus aureus, o s.aureus, hay que recordar que estará todo en minúscula) se cambiarán por el nombre sin espacios 
(staphylococcusaureus o staphylococcus_aureus, es sencillo de lograr de cualquier método) para que al tokenizar sea un solo token. 


In [36]:
dmp = urllib.request.urlopen("https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxcat.zip")
print(len(list(dmp)))


search = Entrez.efetch(db='ScientificName',d=tax, rettype='xml')
result = Entrez.read(search)

for organism in result:
    scientific_name = organism.get("ScientificName")
    print(scientific_name)

24734


HTTPError: HTTP Error 400: Bad Request

# Generación del corpora con Entrez

In [8]:
# Se establecen términos de búsqueda

term = "digimon"

Entrez.max_tries=1
Entrez.email=""

In [9]:
# Búsqueda inicial para comprobar cantidad de coincidencias

scout_search = Entrez.esearch(db="pubmed", rettype="count", term = term)
scout_result = Entrez.read(scout_search)
id_quantity = int(scout_result["Count"])
print(f"{id_quantity} artículo(s) encontrados en PubMed.")

2 artículo(s) encontrados en PubMed.


In [10]:
# Rondas de esearch dividido en rondas de longitud máxima (100k búsquedas)
if id_quantity > 100000:
    rounds = int(id_quantity/100000)+1
    retmax=100*1000
else:
    rounds = 1
    retmax = id_quantity


In [11]:
# Se inicializa la lista de abstracts 
retstart=0
abstracts=[]

# eFetch y adición de los resultados a la lista de abstracts. 
for round in range(0,rounds):
    
    search = Entrez.esearch(db="pubmed", retmax=retmax, term=term)
    
    id_quantity -= retmax
    retstart += retmax
    if id_quantity < 100000:
        retmax = id_quantity
    result = Entrez.read(search)
    

In [12]:
for single_id in result['IdList']:
    search = Entrez.efetch(db="pubmed", id=single_id, rettype="medline", retmode="text")
    record = list(Medline.parse(search))
    record = dict(record[0])
    try:
        record = record["AB"]
        abstracts.append(record)
    except KeyError:
        pass


The role of experience in the development of brain mechanisms for face recognition is intensely debated. Experience with subordinate- and individual-level classification of faces is thought, by some, to be foundational in the development of the specialization of face recognition. Studying children with extremely intense interests (EII) provides an opportunity to examine experience-related changes in non-face object recognition in a population where face expertise is not fully developed. Here, two groups of school-aged children -one group with an EII with Pokemon cards and another group of age-matched controls - underwent fMRI while viewing faces, Pokemon characters, Pokemon objects, and Digimon characters. Pokemon objects were non-character Pokemon cards that experts do not typically individuate during game play and trading. Neither experts nor controls had previous experience with Digimon characters. As expected, experts and controls showed equivalent activation in the fusiform face a

In [13]:
cantidad_inicial=len(abstracts)

print(f'La cantidad inicial de abstracts es {cantidad_inicial}.')

La cantidad inicial de abstracts es 2.


In [ ]:
# Cambiar enfermedad por ID

cantidad_enfermedad=0
enfermedades_detectadas = []

for disease_list in disease_dict.values():
    for disease_name in disease_list:
        for text in abstracts:
            if disease_name in text:
                disease_id = extract_key_from_value(disease_dict, disease_name)
                text.replace(disease_name, disease_id )
                cantidad_enfermedad += 1
                enfermedades_detectadas.append(disease_id)


In [ ]:
cantidad_bacteria = 0
bacterias_detectadas = []


# Tokenización de los abstracts (tokenización bruta pasando a minúscula)
tokenized_abstracts = [single_text.lower().split() for single_text in abstracts]

# Eliminación de signos de puntuación
